In [1]:
# Lets import the libraries
import socket, cv2, pickle, struct, imutils ,threading

In [2]:
def transmit():
    while True:
    #client_socket,addr = server_socket.accept()
    #print('GOT CONNECTION FROM:',addr)
        if client_socket:
            vid = cv2.VideoCapture(0)
            while(vid.isOpened()):
                img,frame = vid.read()
                frame = imutils.resize(frame,width=320)
                a = pickle.dumps(frame)
                message = struct.pack("Q",len(a))+a
                client_socket.sendall(message)           
                cv2.imshow('TRANSMITTING VIDEO FROM lINUX',frame)
                key = cv2.waitKey(1) & 0xFF
                if key ==ord('q'):
                    client_socket.close()
                    break
    cv2.destroyAllWindows()

In [3]:

def recieve(data,payload_size):
    while True:
        while len(data) < payload_size:
            packet = client_socket.recv(4*1024) 
            if not packet: break
            data+=packet
        packed_msg_size = data[:payload_size]
        data = data[payload_size:]
        msg_size = struct.unpack("Q",packed_msg_size)[0]
        while len(data) < msg_size:
            data += client_socket.recv(4*1024)
        frame_data = data[:msg_size]
        data  = data[msg_size:]
        frame = pickle.loads(frame_data)
        cv2.imshow("RECEIVING VIDEO FROM WINDOWS",frame)
        key = cv2.waitKey(1) & 0xFF
        if key  == ord('q'):
            break
    client_socket.close()
    cv2.destroyAllWindows()

In [4]:
# create socket
client_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
host_ip = '192.168.99.1' #IP Address of Host to be Entered 
port = 9999
client_socket.connect((host_ip,port))

In [5]:
data = b""
payload_size = struct.calcsize("Q")
x1=threading.Thread(target=recieve , args=(data,payload_size))
x2=threading.Thread(target=transmit)
x1.start()
x2.start()


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\sharm\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\sharm\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-713de09ac844>", line 4, in recieve
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine
Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\sharm\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\sharm\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-125133c2d273>", line 9, in transmit
  File "C:\Users\sharm\anaconda3\lib\site-packages\imutils\convenience.py", line 69, in resize
    (h, w) = image.shape[:2]
AttributeError: 'NoneType' object has no attribute 'shape'
